Notebook to predict N+1 command where N is 1..4 and command sets can be up to len 5

Construct Graph Function

In [1]:
from graph import Node

def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=1)
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += 1

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, len(session)):

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=1)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = 1
            else:
                child_node.commands[session[cmd]] += 1
            
            cur_node = child_node
        
                
        
    return command_dict


Get Prediction Function

In [16]:
def get_prediction(command_list: list[str], graph, result_size=5, graph_depth=4):
    if len(command_list) == 0:
        return None

    commands = command_list[:graph_depth]
    # should be labeled last command
    last_command = command_list[graph_depth]

    if len(commands) == 0:
        return None, [-1]

    program = commands[0].split()[0]

    frequencies = []
    
    if graph.get(program) is None:
        frequencies.append(-1)
        return None, frequencies
    else:
        node = graph[program]
        frequencies.append(node.frequency)

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
            frequencies.append(node.frequency)
        else:
            frequencies.append(-1)
            return None, frequencies
    
    # last frequency if it has it
    next_program = last_command.split()[0]

    if node.children.get(next_program) is not None:
        frequencies.append(node.children[next_program].frequency)
    else:
        frequencies.append(-1)
    
    return node.get_prediction(previous_command=last_command, num_to_return=result_size), frequencies

Accuracy Function

In [ ]:
from thefuzz import fuzz
from thefuzz import process

from statistics import mean

from tqdm import tqdm

from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def append_list(lst, results, commands, graph_depth):
    lst.append({"Results": results, "Expected": commands[graph_depth], "Command Sequence": commands})

def get_accuracy(command_subsets, fail_list = [], fail_frequencies= [], succeed_list = [], succeed_frequencies = [], correct_5_not_3= [], correct_15_not_5 = [], print_fails=False, graph_depth=4):
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    # filter out nodes that can't get prediction because not enough length so don't contribute and will cause error
    if (graph_depth != -1):
        test_data = [x for x in test_data if len(x) > graph_depth]
    test_size = len(test_data)

    return_3 = 3
    return_5 = 5
    return_15 = 15

    correct = 0
    correct_with_5 = 0
    correct_with_15 = 0
    first_prediction = 0
    has_prediction = 0
    incorrect = 0
    none_count = 0

    for commands in tqdm(test_data, leave=False):
        results_3, frequencies = get_prediction(commands, train_graph, return_3, graph_depth)
        results_5, _ = get_prediction(commands, train_graph, return_5, graph_depth)
        results_15, _ = get_prediction(commands, train_graph, return_15, graph_depth)

        command_freq = [(commands[i], frequencies[i]) for i in range(len(frequencies))]
        
        prev_correct = correct
        prev5_correct =  correct_with_5

        if results_3 is not None:
            has_prediction += 1

            for i in range(len(results_3)):
                if fuzz.ratio(results_3[i][0], commands[graph_depth]) > 85:
                    correct += 1

                    succeed_frequencies.append(mean(frequencies))

                    if correct <= 100:
                        append_list(succeed_list, results_5, command_freq, graph_depth)

                    if i == 0:
                        first_prediction += 1
                
                    break

            if prev_correct == correct:
                fail_frequencies.append(mean(frequencies))
                incorrect += 1

            for i in range(len(results_5)):
                if fuzz.ratio(results_5[i][0], commands[graph_depth]) > 85:
                    correct_with_5 += 1

                    if correct_with_5 - correct <= 100 and prev_correct == correct:
                        append_list(correct_5_not_3, results_5, command_freq, graph_depth)

                    break

            for i in range(len(results_15)):
                if fuzz.ratio(results_15[i][0], commands[graph_depth]) > 85:
                    correct_with_15 += 1

                    if correct_with_15 - correct <= 100 and prev5_correct == correct_with_5:
                        append_list(correct_15_not_5, results_15, command_freq, graph_depth)

                    break

        else:
            none_count += 1

        if results_5 is not None and prev_correct == correct and print_fails:
            append_list(fail_list, results_5, command_freq, graph_depth)  
                      
    return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}% |\n Average Success Freq: {:.0f} |\n Average Fail Freq: {:.0f}'.format(100 * correct/(test_size-none_count), 100 * correct_with_5/(test_size-none_count), 100 * correct_with_15/(test_size-none_count), 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size, mean(succeed_frequencies)/len(command_subsets), mean(fail_frequencies)/len(command_subsets))
    # return 'Correct Proportion: {:.2f}% |\n Correct in 5 not 3: {:.2f}% |\n Correct in 15 not 5: {:.2f}% |\n Has Prediction and is Correct: {:.2f}% |\n Incorrect Proportion: {:.2f}% |\n None Proportion: {:.2f}% |\n First Prediction: {:.2f}% |\n Average Success Freq: {:.0f} |\n Average Fail Freq: {:.0f}'.format(100 * correct/test_size, 100 * correct_with_5/test_size, 100 * correct_with_15/test_size, 100 * correct/has_prediction, 100 * incorrect/test_size, 100 * none_count/test_size, 100 * first_prediction/test_size, mean(succeed_frequencies), mean(fail_frequencies))

In [4]:
from parse import Parser

parser = Parser()

100%|██████████| 56/56 [00:06<00:00,  8.78it/s]


All Commands Parse and Replace Args

In [5]:
subset_size = 5

science_session = parser.parse_commands_per_session(parser.scientists_files)
science_session = parser.parse_commands_into_subsets_sliding_window(science_session, subset_size)
science_session = parser.replace_args_nested(science_session)

experienced_session = parser.parse_commands_per_session(parser.experienced_files)
experienced_session = parser.parse_commands_into_subsets_sliding_window(experienced_session, subset_size)
experienced_session = parser.replace_args_nested(experienced_session)

non_session = parser.parse_commands_per_session(parser.non_programmers_files)
non_session = parser.parse_commands_into_subsets_sliding_window(non_session, subset_size)
non_session = parser.replace_args_nested(non_session)

novice_session = parser.parse_commands_per_session(parser.novice_files)
novice_session = parser.parse_commands_into_subsets_sliding_window(novice_session, subset_size)
novice_session = parser.replace_args_nested(novice_session)

100%|██████████| 56/56 [00:06<00:00,  8.48it/s]


Accuracy predicting N+1 command from N commands with all commands in graph

In [9]:
for i in range(1, 5):
    print("Accuracy predicting command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session:\n", get_accuracy(science_session, graph_depth=i), '\n')
    print("Experienced session:\n", get_accuracy(experienced_session, graph_depth=i), '\n')
    print("Non programmer session:\n", get_accuracy(non_session, graph_depth=i), '\n')
    print("Novice session:\n", get_accuracy(novice_session, graph_depth=i), '\n')

Accuracy predicting command 2 from commands 1 through 1:



100%|██████████| 81083/81083 [08:17<00:00, 162.90it/s]


Scientist session:
 Correct Proportion: 61.33% |
 Correct in 5 not 3: 67.76% |
 Correct in 15 not 5: 81.82% |
 Has Prediction and is Correct: 61.33% |
 Incorrect Proportion: 38.66% |
 None Proportion: 0.03% |
 First Prediction: 43.70% |
 Average Success Freq: 8960 |
 Average Fail Freq: 11593 



100%|██████████| 48920/48920 [13:46<00:00, 59.18it/s]   


Experienced session:
 Correct Proportion: 64.94% |
 Correct in 5 not 3: 70.89% |
 Correct in 15 not 5: 83.98% |
 Has Prediction and is Correct: 64.94% |
 Incorrect Proportion: 35.05% |
 None Proportion: 0.03% |
 First Prediction: 46.17% |
 Average Success Freq: 10959 |
 Average Fail Freq: 13340 



...

Experienced session:
 Correct Proportion: 28.95% |
 Correct in 5 not 3: 29.60% |
 Correct in 15 not 5: 30.21% |
 Has Prediction and is Correct: 28.95% |
 Incorrect Proportion: 64.30% |
 None Proportion: 9.49% |
 First Prediction: 22.94% |
 Average Success Freq: 21518 |
 Average Fail Freq: 23475 



100%|██████████| 3525/3525 [00:00<00:00, 5169.47it/s]


Non programmer session:
 Correct Proportion: 39.29% |
 Correct in 5 not 3: 40.62% |
 Correct in 15 not 5: 41.89% |
 Has Prediction and is Correct: 39.29% |
 Incorrect Proportion: 56.91% |
 None Proportion: 6.27% |
 First Prediction: 31.89% |
 Average Success Freq: 21597 |
 Average Fail Freq: 23575 



100%|██████████| 11447/11447 [00:03<00:00, 3380.49it/s]


Novice session:
 Correct Proportion: 59.96% |
 Correct in 5 not 3: 61.18% |
 Correct in 15 not 5: 62.57% |
 Has Prediction and is Correct: 59.96% |
 Incorrect Proportion: 38.14% |
 None Proportion: 4.74% |
 First Prediction: 46.85% |
 Average Success Freq: 22149 |
 Average Fail Freq: 23807 



Consolidated predicting next command. Graph depth of `-1` should accomplish this.

In [17]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    fail_frequencies = []
    succeed_frequencies = []

    print(get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=-1))

    os.makedirs(os.path.dirname("fails/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("fails/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(fail_frequencies)

    os.makedirs(os.path.dirname("successes/frequencies_all/" + dataset_names[i] + ".csv"), exist_ok=True)
    with open("successes/frequencies_all/" + dataset_names[i] + ".csv", "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(succeed_frequencies)


  5%|▌         | 4338/81270 [00:13<03:46, 339.89it/s]

Save files for all data

In [6]:
import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_list = []
        succeed_list = []
        correct_5_not_3 = []
        correct_15_not_5 = []

        get_accuracy(datasets[i], fail_list=fail_list, succeed_list=succeed_list, correct_5_not_3=correct_5_not_3, correct_15_not_5=correct_15_not_5, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("fails/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(fail_list))

        os.makedirs(os.path.dirname("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("successes/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(succeed_list))

        os.makedirs(os.path.dirname("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_5_not_3/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_5_not_3))

        os.makedirs(os.path.dirname("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt"), exist_ok=True)
        with open("correct_15_not_5/predicting_" + str(j+1) + "/" + dataset_names[i] + ".txt", "w") as file:
            file.write(pp.pformat(correct_15_not_5))


Save frequencies for successes and failures to CSV

In [14]:
import os
import csv

dataset_names = ["Science data", "Experienced data", "Non programmer data", "Novice data"]
datasets = [science_session, experienced_session, non_session, novice_session]

for i in range(len(datasets)):
    for j in range (1, 5):
        fail_frequencies = []
        succeed_frequencies = []

        get_accuracy(datasets[i], fail_frequencies=fail_frequencies, succeed_frequencies=succeed_frequencies, print_fails=True, graph_depth=j)

        os.makedirs(os.path.dirname("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("fails/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fail_frequencies)

        os.makedirs(os.path.dirname("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv"), exist_ok=True)
        with open("successes/frequencies_" + str(j+1) + "/" + dataset_names[i] + ".csv", "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(succeed_frequencies)


Sampled Commands Parse and Replace Args

In [ ]:
from random import sample
import math

subset_size = 5

science_session_sample = parser.parse_commands_per_session(sample(parser.scientists_files, math.floor(0.2 * len(parser.scientists_files))))
science_session_sample = parser.parse_commands_into_subsets_sliding_window(science_session_sample, subset_size)
science_session_sample = parser.replace_args_nested(science_session_sample)

experienced_session_sample = parser.parse_commands_per_session(sample(parser.experienced_files, math.floor(0.2 * len(parser.experienced_files))))
experienced_session_sample = parser.parse_commands_into_subsets_sliding_window(experienced_session_sample, subset_size)
experienced_session_sample = parser.replace_args_nested(experienced_session_sample)

non_session_sample = parser.parse_commands_per_session(sample(parser.non_programmers_files, math.floor(0.2 * len(parser.non_programmers_files))))
non_session_sample = parser.parse_commands_into_subsets_sliding_window(non_session_sample, subset_size)
non_session_sample = parser.replace_args_nested(non_session_sample)

novice_session_sample = parser.parse_commands_per_session(sample(parser.novice_files, math.floor(0.2 * len(parser.novice_files))))
novice_session_sample = parser.parse_commands_into_subsets_sliding_window(novice_session_sample, subset_size)
novice_session_sample = parser.replace_args_nested(novice_session_sample)

100%|██████████| 11/11 [00:02<00:00,  5.47it/s]


Accuracy predicting N+1 command from N commands with subset of data

In [ ]:
for i in range(1, 5):
    print("Accuracy Predicting Command {} from commands {} through {}:\n".format(i+1, 1, i))

    print("Scientist session sample:\n", get_accuracy(science_session_sample, graph_depth=i))
    print("Experienced session sample:\n", get_accuracy(experienced_session_sample, graph_depth=i))
    print("Non programmer session sample:\n", get_accuracy(non_session_sample, graph_depth=i))
    print("Novice session sample:\n", get_accuracy(novice_session_sample, graph_depth=i))

  0%|          | 0/4 [00:00<?, ?it/s]

Accuracy Predicting Command 2 from commands 1 through 1:

Scientist session sample:
 Correct Proportion: 68.81% |
 Correct in 15 not 5: 82.54% |
 Has Prediction and is Correct: 68.81% |
 Incorrect Proportion: 31.19% |
 None Proportion: 0.00% |
 First Prediction: 44.93%
Experienced session sample:
 Correct Proportion: 70.46% |
 Correct in 15 not 5: 82.42% |
 Has Prediction and is Correct: 70.46% |
 Incorrect Proportion: 29.54% |
 None Proportion: 0.00% |
 First Prediction: 44.51%
Non programmer session sample:
 Correct Proportion: 72.88% |
 Correct in 15 not 5: 84.91% |
 Has Prediction and is Correct: 72.88% |
 Incorrect Proportion: 27.12% |
 None Proportion: 0.00% |
 First Prediction: 46.52%


 25%|██▌       | 1/4 [14:46<44:18, 886.24s/it]

Novice session sample:
 Correct Proportion: 82.73% |
 Correct in 15 not 5: 92.47% |
 Has Prediction and is Correct: 82.73% |
 Incorrect Proportion: 17.27% |
 None Proportion: 0.00% |
 First Prediction: 51.90%
Accuracy Predicting Command 3 from commands 1 through 2:

Scientist session sample:
 Correct Proportion: 84.20% |
 Correct in 15 not 5: 93.33% |
 Has Prediction and is Correct: 84.21% |
 Incorrect Proportion: 15.79% |
 None Proportion: 0.01% |
 First Prediction: 60.52%
Experienced session sample:
 Correct Proportion: 83.73% |
 Correct in 15 not 5: 92.60% |
 Has Prediction and is Correct: 83.74% |
 Incorrect Proportion: 16.26% |
 None Proportion: 0.01% |
 First Prediction: 61.59%
Non programmer session sample:
 Correct Proportion: 86.02% |
 Correct in 15 not 5: 93.32% |
 Has Prediction and is Correct: 86.02% |
 Incorrect Proportion: 13.98% |
 None Proportion: 0.00% |
 First Prediction: 65.58%


 50%|█████     | 2/4 [15:18<12:47, 383.78s/it]

Novice session sample:
 Correct Proportion: 89.89% |
 Correct in 15 not 5: 96.25% |
 Has Prediction and is Correct: 89.89% |
 Incorrect Proportion: 10.11% |
 None Proportion: 0.00% |
 First Prediction: 66.82%
Accuracy Predicting Command 4 from commands 1 through 3:

Scientist session sample:
 Correct Proportion: 92.72% |
 Correct in 15 not 5: 97.28% |
 Has Prediction and is Correct: 92.72% |
 Incorrect Proportion: 7.28% |
 None Proportion: 0.01% |
 First Prediction: 74.85%
Experienced session sample:
 Correct Proportion: 92.54% |
 Correct in 15 not 5: 96.33% |
 Has Prediction and is Correct: 92.54% |
 Incorrect Proportion: 7.46% |
 None Proportion: 0.00% |
 First Prediction: 76.20%
Non programmer session sample:
 Correct Proportion: 93.66% |
 Correct in 15 not 5: 97.47% |
 Has Prediction and is Correct: 93.66% |
 Incorrect Proportion: 6.34% |
 None Proportion: 0.00% |
 First Prediction: 78.04%


 75%|███████▌  | 3/4 [15:23<03:31, 211.14s/it]

Novice session sample:
 Correct Proportion: 95.06% |
 Correct in 15 not 5: 98.90% |
 Has Prediction and is Correct: 95.06% |
 Incorrect Proportion: 4.94% |
 None Proportion: 0.00% |
 First Prediction: 73.70%
Accuracy Predicting Command 5 from commands 1 through 4:

Scientist session sample:
 Correct Proportion: 81.89% |
 Correct in 15 not 5: 84.08% |
 Has Prediction and is Correct: 82.17% |
 Incorrect Proportion: 17.77% |
 None Proportion: 0.34% |
 First Prediction: 69.53%
Experienced session sample:
 Correct Proportion: 81.36% |
 Correct in 15 not 5: 83.26% |
 Has Prediction and is Correct: 81.59% |
 Incorrect Proportion: 18.35% |
 None Proportion: 0.29% |
 First Prediction: 71.52%
Non programmer session sample:
 Correct Proportion: 85.71% |
 Correct in 15 not 5: 87.88% |
 Has Prediction and is Correct: 85.90% |
 Incorrect Proportion: 14.07% |
 None Proportion: 0.22% |
 First Prediction: 74.03%


100%|██████████| 4/4 [15:25<00:00, 231.35s/it]

Novice session sample:
 Correct Proportion: 90.95% |
 Correct in 15 not 5: 92.81% |
 Has Prediction and is Correct: 91.05% |
 Incorrect Proportion: 8.94% |
 None Proportion: 0.11% |
 First Prediction: 73.83%
